# **Name - PRABHAKAR KUMAR**
# **Roll - IRM2017008**
# **Naive Bayes Assignment Part 2 Using Bernoulli**

In [27]:
import numpy as np
import pandas as pd
from collections import defaultdict
import string

#**Loading Dataset and preparing Testing and Training Data**

In [28]:
df = pd.read_csv("spam ham data set.csv",encoding='latin-1')
df.dropna(axis=1, inplace=True)
df.head

<bound method NDFrame.head of         v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [29]:
row_count = len(df.index)
train_count = int(0.7*row_count)
print(train_count)

3900


In [30]:
trainSet = df[:3900]
testSet = df[3900:]
testSet.reset_index(inplace=True,drop=True)

In [32]:
print(trainSet['v2'][0])
print(trainSet['v1'][0])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham


# **Creating List of Words and dropping Stop Words**

In [33]:
global_word_list = []

spam_count = 0
ham_count = 0

for index,row in trainSet.iterrows():

    if(row['v1']=='spam'):
        spam_count += 1
    else:
        ham_count += 1

    # Splitting each text content into list of words
    wordList = str(row['v2']).split()

    # Removing Punctuations marks and lower casing from the words as they have no significance towards the model
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]

    # Removing empty words, as some empty words after removal of punctuations may have occured
    wordList = [word for word in wordList if word != '']

    for word in wordList:
        if word not in global_word_list:
            global_word_list.append(word)

print(global_word_list)    

['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat', 'ok', 'lar', 'joking', 'wif', 'u', 'oni', 'free', 'entry', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', '87121', 'receive', 'questionstd', 'txt', 'ratetcs', 'apply', '08452810075over18s', 'dun', 'say', 'so', 'early', 'hor', 'c', 'already', 'then', 'nah', 'i', 'dont', 'think', 'he', 'goes', 'usf', 'lives', 'around', 'here', 'though', 'freemsg', 'hey', 'darling', 'its', 'been', '3', 'weeks', 'now', 'and', 'no', 'word', 'back', 'id', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'xxx', 'std', 'chgs', 'send', 'å£150', 'rcv', 'even', 'my', 'brother', 'is', 'not', 'speak', 'with', 'me', 'they', 'treat', 'aids', 'patent', 'as', 'per', 'your', 'request', 'melle', 'oru', 'minnaminunginte', 'nurungu', 'vettam', 'has', 'set', 'callertune', 'all', 'callers', 'press

In [34]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from nltk.corpus import stopwords  

stop_words = set(stopwords.words('english'))

In [36]:
print(stop_words)

{'of', 'than', 'who', 'such', 'yours', "should've", 'their', 've', "haven't", 'you', 'whom', 'after', 'more', 'to', 'did', 'its', 'o', 'into', "hadn't", 'doesn', 'myself', 'further', 'and', 'same', 'not', 'd', "weren't", "couldn't", 'aren', 'i', 'but', 'down', 'then', 'haven', "shan't", 'until', 'where', 'other', 'what', "wouldn't", "you'd", 'll', 'ourselves', 'all', 'doing', 'by', 're', 'hasn', 'most', "won't", 'an', 'm', 'we', 'wasn', 'if', "mustn't", 'up', 'this', "aren't", 'in', "you've", 'was', 'out', 'very', 'don', 'mustn', 'her', 'can', 'those', 'weren', 'so', "shouldn't", 'no', "doesn't", "isn't", 'my', 'under', 'once', 'am', 'because', 'before', 'shan', 'with', 'were', "that'll", 't', "wasn't", 'ain', 'over', 'yourselves', "it's", 'why', 'as', "hasn't", "needn't", 'him', 'few', 'now', 'or', 'while', 'both', 'about', 'your', 'should', 'me', 'below', 'some', 'against', 'again', 'be', 'any', 'mightn', 'that', 'being', 'himself', 'only', 'do', 'been', 'there', 'having', 'them', 'w

In [37]:
iter=0
word_token = {}

for word in global_word_list:
  if word in stop_words:
    continue
  word_token[word] = iter
  iter += 1


# **Converting Input Mails to list of Bernoulli Variables**

In [38]:
data = []
for index,row in trainSet.iterrows():
    wordList = str(row['v2']).split()
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]
    wordList = [word for word in wordList if word != '']
    temp_data = []
    for word in word_token:
        if word in wordList:
            temp_data.append(1)
        else:
            temp_data.append(0)
    data.append(temp_data)


In [39]:
print(data[2])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [40]:
print(len(data))

3900


In [41]:
print(len(data[0]))

7732


# **Calculating Class Probabilithy for Each words**

In [43]:
#spam = 1
#ham = 0

row_count = len(data)

word_spam_map = {}
word_ham_map = {}

for word in word_token:
    word_index = word_token[word]

    temp_spam_count = 0
    temp_ham_count = 0
    
    for i in range(row_count):
        if(data[i][index] == 0):
            continue
        if(trainSet['v1'][i] == 'spam'):
            temp_spam_count += 1
        else:
            temp_ham_count += 1
    
    word_spam_map[word] = (temp_spam_count+1)/(spam_count+2)
    word_ham_map[word] = (temp_ham_count+1)/(ham_count+2)

In [44]:
pSpam = spam_count / (spam_count + ham_count)
pHam = ham_count / (spam_count + ham_count)

# **Probability of Mail being Spam**

In [45]:
def spamProb(wordList):
    wordSpamProb = []
    for word in word_token:
        if word in wordList:
            wordSpamProb.append(word_spam_map[word])
        else:
            wordSpamProb.append(1-word_spam_map[word])
    prob = pSpam
    for item in wordSpamProb:
        prob*=item
    return prob

# **Probability of Mail being Ham**

In [46]:
def hamProb(wordList):
    wordHamProb = []
    for word in word_token:
        if word in wordList:
            wordHamProb.append(word_ham_map[word])
        else:
            wordHamProb.append(1-word_ham_map[word])
    prob = pHam
    for item in wordHamProb:
        prob*=item
    return prob

# **Testing of Model over Test Data**

In [47]:
actual = np.zeros(len(testSet.index), dtype=int)
predicted = np.zeros(len(testSet.index), dtype=int)
for index, row in testSet.iterrows():
    if str(row['v1'])=='spam':
        actual[index] = 1
    else:
        actual[index] = 0
    
    mail = str(row['v2'])
    wordList = str(mail).split()
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]
    wordList = [word for word in wordList if word != '']
    
    pSpamMail = spamProb(wordList)
    pHamMail = hamProb(wordList)
    
    if pSpamMail > pHamMail:
        predicted[index] = 1
    else:
        predicted[index] = 0
        
print('TOTAL SAMPLES : ' + str(len(testSet.index)))
print('SAMPLES CLASSIFIED CORRECTLY : ' + str(np.sum(actual == predicted)))
print('SAMPLES CLASSIFIED INCORRECTLY : ' + str(len(testSet.index) - np.sum(actual == predicted)))
print('MODEL ACCURACY : ' + str(((np.sum(actual == predicted))/(len(testSet.index)))*100))

TOTAL SAMPLES : 1672
SAMPLES CLASSIFIED CORRECTLY : 1324
SAMPLES CLASSIFIED INCORRECTLY : 348
MODEL ACCURACY : 79.18660287081339


# **Accuracy Parameters of the Model over Test Data**

In [48]:
tp=0
tn=0
fp=0
fn=0
for i in range(1672):
    if actual[i]==predicted[i]and actual[i]==0:
        tp+=1
    if actual[i]!=predicted[i]and actual[i]==0:
        fp+=1
    if actual[i]==predicted[i]and actual[i]==1:
        tn+=1
    if actual[i]!=predicted[i]and actual[i]==1:
        fn+=1

print("CONFUSION MATRIX")
print(tp," ",fp)
print(fn," ",tn)

CONFUSION MATRIX
1152   292
56   172


In [49]:
print("Accuracy : ",str((tp+tn)/(tp+fp+tn+fn)))
print("Precision : ",str((tp)/(tp+fp)))
print("Sensitivity : ",str((tp)/(tp+fn)))
print("Specificity : ",str((tn)/(fp+tn)))

Accuracy :  0.7918660287081339
Precision :  0.7977839335180056
Sensitivity :  0.9536423841059603
Specificity :  0.3706896551724138
